Transfer learning using cryptobert and roberta

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import pandas as pd
from scipy.special import softmax

/Users/endy/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torch

In [3]:
!pip install protobuf==3.20.0

## Step 1 - Importing a sample of some Bitcoin Tweet Data from Kaggle to begin analysing the model

In [4]:
data = pd.read_csv('~/code/giadapi/crypto/data/raw/bitcoin_tweets1000000.csv', nrows = 1000)

# "\\wsl.localhost\Ubuntu\home\peter\code\giadapi\crypto\kaggle-tweets.zip"

In [5]:
data

,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,cleanText,Polarity Score,sentiment
0,0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False,blue ridge bank shares halted nyse bitcoin atm...,0.00,0
1,1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False,today thursday take friend leowandersleb btc w...,0.00,0
2,2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False,guys evening read article btc would like share,0.00,0
3,3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False,btc big chance billion price bitcoin btc crypto,0.00,0
4,4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False,network secured nodes today soon biggest bears...,-0.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Szymon Kostrzewa,NaN,NaN,2016-04-02 12:33:50,7.0,75,31,False,2021-02-10 17:27:33,🤝 Follow me on @betfury_io. Let's hunt for Bit...,NaN,Twitter Web App,False,follow betfury_io let hunt bitcoins together g...,0.30,1
996,996,Crypto Zombie,New York,🎥 YouTube: Crypto Zombie 🧟‍♂️ #Bitcoin and #bl...,2010-01-18 23:52:55,59416.0,1058,23385,False,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,['Bitcoin'],Twitter Web App,False,bitcoin falling tesla pump top btc worried dip...,0.50,1
997,997,Brian Harrington,"Anaheim, CA",#Bitcoin is actively replacing the dollar. Can...,2009-08-19 20:21:01,7687.0,2322,83966,False,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,NaN,Twitter Web App,False,within driving distance iowa city definitely c...,0.00,0
998,998,🔸️Bitcoin Pixel🔸️,NaN,I discovered crypto in 2019 on Youtube. Since ...,2015-07-08 13:44:42,1873.0,75,676,False,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,"['BTC', 'ETH', 'DOT']",Twitter for Android,False,question please finally sold avax amp cake bou...,0.00,0


In [6]:
data = data[['user_name','text', 'date']]

In [7]:
data['text'][0]

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement https://t.co/xaaZmaJKiV @MyBlueRidgeBank… https://t.co/sgBxMkP1SI'

## Step 2 - Cleaning the data

In [8]:
# Preprocess text (username and link placeholders) - this is suggested from the example in 
# https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment
def preprocess_1(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

preprocess_1(data['text'][0])

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement http @user http'

In [9]:
# I have changed this to remove more information

def preprocess_2(text):
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

sample_test = preprocess_2(data['text'][0])
sample_test

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement   '

In [10]:
#use the preprocess_2 to clean the data

data['process_text'] = data.text
data['label_xlm'] = data.text
data['score_xlm'] = data.text
data['label_bert'] = data.text
data['score_bert'] = data.text
for i in range(len(data)):
    data['process_text'][i] = preprocess_2(data['text'][i])

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/1361758759.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['process_text'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/1361758759.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label_xlm'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/1361758759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

## Step 3 - analyse the language and sentiments by pretrained model

In [11]:
!pyenv local crypto

In [12]:
#Model 1 - Use the xlm model - https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment?text=go+up

from transformers import pipeline
model_path_xlm = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task_xlm = pipeline("sentiment-analysis", model=model_path_xlm, tokenizer=model_path_xlm)

In [13]:
#Model 2 - Use the CryptoBERT model - https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment?text=go+up

from transformers import pipeline
model_path_bert = "ElKulako/cryptobert"
sentiment_task_bert = pipeline("sentiment-analysis", model=model_path_bert, tokenizer=model_path_bert)

In [14]:
test_sent = sentiment_task_xlm(sample_test)
print(test_sent[0]['score'])
print(test_sent[0]['label'])

0.6874694228172302
negative


In [15]:
test_sent = sentiment_task_bert(sample_test)
print(test_sent[0]['score'])
print(test_sent[0]['label'])

0.9938846230506897
Neutral


In [16]:
for i in range(len(data)):
    sentiment_xlm = sentiment_task_xlm(data['process_text'][i])
    data['label_xlm'][i] = sentiment_xlm[0]['label']
    data['score_xlm'][i] = sentiment_xlm[0]['score']
    sentiment_bert = sentiment_task_bert(data['process_text'][i])
    data['label_bert'][i] = sentiment_bert[0]['label']
    data['score_bert'][i] = sentiment_bert[0]['score']

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/2870314635.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label_xlm'][i] = sentiment_xlm[0]['label']
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/2870314635.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score_xlm'][i] = sentiment_xlm[0]['score']
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/2870314635.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [17]:
data

,user_name,text,date,process_text,label_xlm,score_xlm,label_bert,score_bert
0,DeSota Wilson,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,negative,0.687469,Neutral,0.993885
1,CryptoND,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...",neutral,0.65865,Neutral,0.849627
2,Tdlmatias,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"Guys evening, I have read this article about B...",positive,0.55389,Bearish,0.780925
3,Crypto is the future,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,neutral,0.698535,Bullish,0.653737
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,neutral,0.530071,Bullish,0.907657
...,...,...,...,...,...,...,...,...
995,Szymon Kostrzewa,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2021-02-10 17:27:33,🤝 Follow me on Let's hunt for Bitcoins togeth...,positive,0.572492,Neutral,0.62777
996,Crypto Zombie,#Bitcoin falling after Tesla pump! Was that th...,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,neutral,0.515613,Bullish,0.630188
997,Brian Harrington,If you're within driving distance of Iowa City...,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,positive,0.570308,Neutral,0.616015
998,🔸️Bitcoin Pixel🔸️,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,neutral,0.828953,Neutral,0.617814


## Step 4 - compare the model performance

In [18]:
def standardise(x):
    if (x == 'Neutral'):
        return 'neutral'
    elif (x == 'Bullish'):
        return 'positive'
    else:
        return 'negative'
data["label_bert"] = data["label_bert"].apply(standardise)

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/1388689369.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["label_bert"] = data["label_bert"].apply(standardise)


In [19]:
data

,user_name,text,date,process_text,label_xlm,score_xlm,label_bert,score_bert
0,DeSota Wilson,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,negative,0.687469,neutral,0.993885
1,CryptoND,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...",neutral,0.65865,neutral,0.849627
2,Tdlmatias,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"Guys evening, I have read this article about B...",positive,0.55389,negative,0.780925
3,Crypto is the future,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,neutral,0.698535,positive,0.653737
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,neutral,0.530071,positive,0.907657
...,...,...,...,...,...,...,...,...
995,Szymon Kostrzewa,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2021-02-10 17:27:33,🤝 Follow me on Let's hunt for Bitcoins togeth...,positive,0.572492,neutral,0.62777
996,Crypto Zombie,#Bitcoin falling after Tesla pump! Was that th...,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,neutral,0.515613,positive,0.630188
997,Brian Harrington,If you're within driving distance of Iowa City...,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,positive,0.570308,neutral,0.616015
998,🔸️Bitcoin Pixel🔸️,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,neutral,0.828953,neutral,0.617814


In [20]:
data['label_xlm'].equals(data['label_bert']) 

False

In [21]:
data['equality'] = data['label_xlm'] == data['label_bert']

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/2772664740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['equality'] = data['label_xlm'] == data['label_bert']


In [22]:
data

,user_name,text,date,process_text,label_xlm,score_xlm,label_bert,score_bert,equality
0,DeSota Wilson,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,negative,0.687469,neutral,0.993885,False
1,CryptoND,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...",neutral,0.65865,neutral,0.849627,True
2,Tdlmatias,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"Guys evening, I have read this article about B...",positive,0.55389,negative,0.780925,False
3,Crypto is the future,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,neutral,0.698535,positive,0.653737,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,neutral,0.530071,positive,0.907657,False
...,...,...,...,...,...,...,...,...,...
995,Szymon Kostrzewa,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2021-02-10 17:27:33,🤝 Follow me on Let's hunt for Bitcoins togeth...,positive,0.572492,neutral,0.62777,False
996,Crypto Zombie,#Bitcoin falling after Tesla pump! Was that th...,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,neutral,0.515613,positive,0.630188,False
997,Brian Harrington,If you're within driving distance of Iowa City...,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,positive,0.570308,neutral,0.616015,False
998,🔸️Bitcoin Pixel🔸️,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,neutral,0.828953,neutral,0.617814,True


In [23]:
data['label_xlm'].value_counts()

neutral     710
positive    199
negative     91
Name: label_xlm, dtype: int64

In [24]:
data['label_bert'].value_counts()

neutral     601
positive    346
negative     53
Name: label_bert, dtype: int64

In [25]:
data['equality'].value_counts()

True     561
False    439
Name: equality, dtype: int64

## Step 5: Count the number of positive value by date

In [34]:
data['date'] = pd.to_datetime(data.date)
data['new_date'] = data['date'].dt.date

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/778902647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data.date)
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72969/778902647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['new_date'] = data['date'].dt.date


In [33]:
data

,user_name,text,date,process_text,label_xlm,score_xlm,label_bert,score_bert,equality,new_date
0,DeSota Wilson,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,negative,0.687469,neutral,0.993885,False,2021-02-10
1,CryptoND,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...",neutral,0.65865,neutral,0.849627,True,2021-02-10
2,Tdlmatias,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"Guys evening, I have read this article about B...",positive,0.55389,negative,0.780925,False,2021-02-10
3,Crypto is the future,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,neutral,0.698535,positive,0.653737,False,2021-02-10
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,neutral,0.530071,positive,0.907657,False,2021-02-10
...,...,...,...,...,...,...,...,...,...,...
995,Szymon Kostrzewa,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2021-02-10 17:27:33,🤝 Follow me on Let's hunt for Bitcoins togeth...,positive,0.572492,neutral,0.62777,False,2021-02-10
996,Crypto Zombie,#Bitcoin falling after Tesla pump! Was that th...,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,neutral,0.515613,positive,0.630188,False,2021-02-10
997,Brian Harrington,If you're within driving distance of Iowa City...,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,positive,0.570308,neutral,0.616015,False,2021-02-10
998,🔸️Bitcoin Pixel🔸️,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,neutral,0.828953,neutral,0.617814,True,2021-02-10


In [53]:
df_label_xlm = pd.DataFrame(data.groupby(['new_date', 'label_xlm']).size())

In [50]:
df_label_xlm

0
new_date   label_xlm     
2021-02-10 negative    91
           neutral    710
           positive   199

In [49]:
df_label_xlm.pivot(index='label_xlm',columns='')

KeyError: "None of ['label_xlm', ''] are in the columns"

In [ ]:
df_label_xlm.

In [43]:
df_label_bert= pd.DataFrame(data.groupby(['new_date', 'label_bert']).size())

In [44]:
df_label_bert

0
new_date   label_bert     
2021-02-10 negative     53
           neutral     601
           positive    346

In [101]:
# data

,user_name,text,date,process_text,label_xlm,score_xlm,label_bert,score_bert,equality
0,DeSota Wilson,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,negative,0.687469,neutral,0.993885,False
1,CryptoND,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10,"😎 Today, that's this #Thursday, we will do a ""...",neutral,0.65865,neutral,0.849627,True
2,Tdlmatias,"Guys evening, I have read this article about B...",2021-02-10,"Guys evening, I have read this article about B...",positive,0.55389,negative,0.780925,False
3,Crypto is the future,$BTC A big chance in a billion! Price: \487264...,2021-02-10,$BTC A big chance in a billion! Price: \487264...,neutral,0.698535,positive,0.653737,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,This network is secured by 9 508 nodes as of t...,2021-02-10,This network is secured by 9 508 nodes as of t...,neutral,0.530071,positive,0.907657,False
...,...,...,...,...,...,...,...,...,...
995,Szymon Kostrzewa,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2021-02-10,🤝 Follow me on Let's hunt for Bitcoins togeth...,positive,0.572492,neutral,0.62777,False
996,Crypto Zombie,#Bitcoin falling after Tesla pump! Was that th...,2021-02-10,#Bitcoin falling after Tesla pump! Was that th...,neutral,0.515613,positive,0.630188,False
997,Brian Harrington,If you're within driving distance of Iowa City...,2021-02-10,If you're within driving distance of Iowa City...,positive,0.570308,neutral,0.616015,False
998,🔸️Bitcoin Pixel🔸️,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,2021-02-10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,neutral,0.828953,neutral,0.617814,True


In [102]:
# data.dtypes

user_name       object
text            object
date            object
process_text    object
label_xlm       object
score_xlm       object
label_bert      object
score_bert      object
equality          bool
dtype: object

In [56]:
# data.to_csv('score.csv')